# 🗺️ Explore Database 
In this notebook, `phase 2` I will explore the data, understand the schema and duplication

Data taken from : [Instacart](https://www.kaggle.com/c/instacart-market-basket-analysis/data)

In [2]:
import pandas as pd
import os
import sqlite3
import duckdb
from rankmc.data.etl import datafile_path_finder

In [3]:
data_dir = datafile_path_finder('')
os.chdir(data_dir)

In [4]:
conn = duckdb.connect("instacart.sqlite")

# SQL Schema
### Table Names

In [13]:
conn.execute("SELECT name FROM sqlite_master WHERE type='table'")

# Fetch all the results
table_names = conn.fetchall()

# Print the table names
for table in table_names:
    print(table[0])

aisles
departments
orders
order_products
products


### Phase 1 EDA - Tables

In [6]:
conn.query("SELECT COUNT(product_id) AS cnt FROM products")

┌───────┐
│  cnt  │
│ int64 │
├───────┤
│ 49688 │
└───────┘

In [9]:
products = pd.read_sql('select * from products',con=conn)
products.sample(3)

C:\Users\Forcessofnature\AppData\Local\Temp\ipykernel_25948\2522833535.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  products = pd.read_sql('select * from products',con=conn)


,product_id,product_name,aisle_id,department_id
38026,38027,"Easter Candy Coated Royal Dark Chocolate Eggs,...",45,19
23433,23434,Premium 7 Sources Oil Blend,47,11
34609,34610,Miso Ramen Broth,69,15


In [10]:
orders = pd.read_sql('select * from orders',con=conn)
orders.sample(3)

C:\Users\Forcessofnature\AppData\Local\Temp\ipykernel_25948\4231992749.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  orders = pd.read_sql('select * from orders',con=conn)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
975782,769699,58653,prior,5,5,13,0.0
840550,3254371,50503,prior,1,4,15,NaN
1880921,664438,112904,prior,3,0,22,9.0


In [11]:
order_products = pd.read_sql('select * from order_products',con=conn)
order_products.sample(3)

C:\Users\Forcessofnature\AppData\Local\Temp\ipykernel_25948\1676685220.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  order_products = pd.read_sql('select * from order_products',con=conn)


,order_id,product_id,add_to_cart_order,reordered
30085677,3173497,37646,2,1
27692523,2920356,34160,19,1
6874626,725691,5959,4,1


In [12]:
departments = pd.read_sql('select * from departments',con=conn)
departments.sample(3)

C:\Users\Forcessofnature\AppData\Local\Temp\ipykernel_25948\339927383.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  departments = pd.read_sql('select * from departments',con=conn)


,department_id,department
15,16,dairy eggs
1,2,other
8,9,dry goods pasta


### Explore data types and check for duplicated keys (IDs)

In [14]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [15]:
# Check that all product table has not duplicate keys
sum(products['product_id'].value_counts()>1)

0

In [16]:
departments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   department_id  21 non-null     int64 
 1   department     21 non-null     object
dtypes: int64(1), object(1)
memory usage: 468.0+ bytes


In [17]:
# Check that departments table has not duplicate keys
sum(departments['department_id'].value_counts()>1)

0

In [18]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       object 
 6   days_since_prior_order  float64
dtypes: float64(1), int64(4), object(2)
memory usage: 182.7+ MB


In [23]:
# Check that orders table has not duplicate keys
sum(orders['order_id'].value_counts()>1)

0

In [22]:
# Check that users we have different customers, for orders
sum(orders['user_id'].value_counts()>1)

206209

In [24]:
order_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33819106 entries, 0 to 33819105
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
 3   reordered          int64
dtypes: int64(4)
memory usage: 1.0 GB


### Define Relationships

orders (order_id) <-* order products(order_id) <br>
order products(product_id) *-> proucts(product_id) <br>
products (department_id)*-> departments(department_id)

In [25]:
# check for reordered products
order_products[order_products['reordered'] == 0]

,order_id,product_id,add_to_cart_order,reordered
2,2,9327,3,0
4,2,30035,5,0
8,2,43668,9,0
17,4,46842,1,0
51,5,47209,22,0
...,...,...,...,...
33819084,3421049,26800,5,0
33819085,3421049,34243,6,0
33819089,3421056,16475,4,0
33819090,3421056,12432,5,0
